This notebook reads the raw source file of data logs from a single day, extract the data of interest, followed by a cleaning to finally transform the data to an end dataframe that is exported to a parquet file for later analysis.

### Importing

The first step is to import the pandas library, and create the dataframe with the raw logs. Each line of the raw file is in json format.

In [16]:
import pandas as pd

LOG_FILE = "game_overs_2014_05_21.txt"
df = pd.read_json(LOG_FILE, lines=True)
df.shape

(914823, 59)

The resulting dataframe df is very large (914,823 rows and 59 columns), it can be reduced by filtering out some rows and taking the columns of interest.

### Slicing
First let's take logs from only the version 1.0 of the game.

In [17]:
# work with only the version 1.0 of the application.
df['version'] = df['ver'].astype(str)
df =  df[df['version'] == "1.0"]

The scope of this project cover only the first 50 levels of the game, it is also of not interest to keep the logs where users quit the level before any conclusion.

In [18]:
# work with only the first 50 levels
df = df[df['level_number'] <= 50]

# remove rows with end_reason equal to "quit"
df = df[~df['end_reason'].isin(['quit'])]

Finally, only the columns of interest are selected and the rest are discarded.

In [19]:
# remove columns that do not apply to the current version or that has debug/ dev data
df =df[['device_id', 'ts', 'level_number', 'end_reason', 'lives_left','n_bananavalue', 'n_distance',
        'n_specialcollectibles', 'n_swings', 'swings_left', 'time_used', 'pathtrace', 'stars', 'score']]
df.shape

(838654, 14)

After the slicing process the df was reduced to 838654 rows and 14 columns of interest. Now it can be cleaned

### Cleaning
The very first step is to remove duplicate rows.

In [20]:
# get duplicated rows
mask = df.duplicated()
duplicated_rows = df[mask]
duplicated_rows.shape[0]

56093

There are 56093 duplicated rows that can be removed.

In [21]:
df = df.drop_duplicates()
df.shape

(782561, 14)

After removing duplicate rows the dataframe is reduced to 782561 rows.

Now let's check for missing values

In [22]:
# check for NaN values
df.isna().sum()

device_id                0
ts                       0
level_number             0
end_reason               0
lives_left               0
n_bananavalue            0
n_distance               0
n_specialcollectibles    1
n_swings                 1
swings_left              0
time_used                0
pathtrace                1
stars                    0
score                    1
dtype: int64

There is exactly one missing value in columns: 'n_specialcollectibles','n_swings','pathtrace' and 'score'. 
Let's find out where are those missing values by selecting the rows with NaN values in them.

In [23]:
df[df.isna().any(axis=1)]

,device_id,ts,level_number,end_reason,lives_left,n_bananavalue,n_distance,n_specialcollectibles,n_swings,swings_left,time_used,pathtrace,stars,score
443168,bb046205,1400653973,6,win,5,0,180,NaN,NaN,2,26,NaN,3,NaN


There is only one row with all the NaN values, after checking that this attempt ended in a "win" condition, but it does not have score, tracak path. It was decided to drop the row from the dataframe as the data is invalid.

In [24]:
df = df[~df.isna().any(axis=1)] #using df.dropna(axis=0) would also work.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 782560 entries, 0 to 914822
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   device_id              782560 non-null  object 
 1   ts                     782560 non-null  int64  
 2   level_number           782560 non-null  int64  
 3   end_reason             782560 non-null  object 
 4   lives_left             782560 non-null  int64  
 5   n_bananavalue          782560 non-null  int64  
 6   n_distance             782560 non-null  int64  
 7   n_specialcollectibles  782560 non-null  float64
 8   n_swings               782560 non-null  float64
 9   swings_left            782560 non-null  int64  
 10  time_used              782560 non-null  int64  
 11  pathtrace              782560 non-null  object 
 12  stars                  782560 non-null  int64  
 13  score                  782560 non-null  float64
dtypes: float64(3), int64(8), object(3)
memory

Some columns are float type, but they must be of integer type. This could be caused because Numpy defines the NaN values as float type.

In [25]:
df['n_specialcollectibles'] = df['n_specialcollectibles'].astype(int)
df['n_swings'] = df['n_swings'].astype(int)
df['score'] = df['score'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 782560 entries, 0 to 914822
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   device_id              782560 non-null  object
 1   ts                     782560 non-null  int64 
 2   level_number           782560 non-null  int64 
 3   end_reason             782560 non-null  object
 4   lives_left             782560 non-null  int64 
 5   n_bananavalue          782560 non-null  int64 
 6   n_distance             782560 non-null  int64 
 7   n_specialcollectibles  782560 non-null  int64 
 8   n_swings               782560 non-null  int64 
 9   swings_left            782560 non-null  int64 
 10  time_used              782560 non-null  int64 
 11  pathtrace              782560 non-null  object
 12  stars                  782560 non-null  int64 
 13  score                  782560 non-null  int64 
dtypes: int64(11), object(3)
memory usage: 89.6+ MB


Now let's check the data distribution to look for outliers

In [26]:
df.describe()

,ts,level_number,lives_left,n_bananavalue,n_distance,n_specialcollectibles,n_swings,swings_left,time_used,stars,score
count,7.825600e+05,782560.000000,7.825600e+05,782560.000000,782560.000000,782560.000000,782560.000000,782560.000000,782560.000000,782560.000000,782560.000000
mean,1.400738e+09,12.461266,6.860802e+04,79.075510,283.841153,0.035050,9.229953,0.512082,38.000164,1.291403,1314.369662
std,6.094907e+06,9.551360,1.213764e+07,141.406044,223.004744,0.387486,7.309185,1.512776,29.732880,1.435500,1507.377595
min,1.049000e+03,1.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.400643e+09,5.000000,2.000000e+00,0.000000,93.000000,0.000000,2.000000,0.000000,10.000000,0.000000,100.000000
50%,1.400653e+09,10.000000,4.000000e+00,0.000000,232.000000,0.000000,8.000000,0.000000,31.000000,0.000000,940.000000
75%,1.400660e+09,18.000000,5.000000e+00,115.000000,425.000000,0.000000,14.000000,0.000000,61.000000,3.000000,1816.000000
max,2.095238e+09,50.000000,2.147484e+09,2818.000000,2118.000000,10.000000,60.000000,21.000000,898.000000,3.000000,28229.000000


There are some invalid values in the "lives_left" and "time_used" columns. Let's remove those rows for the dataframe.

In [27]:
# remove outliers
df = df[df['lives_left'] <= 50]
df = df[df['time_used'] <= 200]

### Transforming
One important step is to select only the observations (rows) that happened the specific day of interest for this project (2014-05-21). First the columns 'ts' needs to be converted into datetime type in order to filter the date.

In [28]:
# convert timestamp to datetime
df['date_time'] = pd.to_datetime(df['ts'], unit='s')

# remove rows with wrong date
df = df[df['date_time'].dt.date == pd.to_datetime('2014-05-21').date()]

# remove the timestamp column form the df
df = df.drop(['ts'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 711272 entries, 22 to 914822
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   device_id              711272 non-null  object        
 1   level_number           711272 non-null  int64         
 2   end_reason             711272 non-null  object        
 3   lives_left             711272 non-null  int64         
 4   n_bananavalue          711272 non-null  int64         
 5   n_distance             711272 non-null  int64         
 6   n_specialcollectibles  711272 non-null  int64         
 7   n_swings               711272 non-null  int64         
 8   swings_left            711272 non-null  int64         
 9   time_used              711272 non-null  int64         
 10  pathtrace              711272 non-null  object        
 11  stars                  711272 non-null  int64         
 12  score                  711272 non-null  int64   

After final inspections, the data looks correct and clean, it is ready for Analysis. It is exported into a parquet file given the size of the dataframe.

In [29]:
df.to_parquet('game_overs_compact.parquet', engine='pyarrow', compression='zstd')